In [3]:
import requests
import json
from bs4 import BeautifulSoup
import re
import concurrent.futures

In [4]:
DOMAIN = "https://www.pinkoi.com"
style_dict = {"可愛":"cute", "簡約":"minimal", "浪漫":"romantic", "復古":"vintage", "雜貨":"zakka", 
         "中性":"neutral", "民族":"green", "都會":"urban", "禪風":"zen"}

In [5]:
resp = requests.get("https://www.pinkoi.com/browse")
soup = BeautifulSoup(resp.text, "html5lib")
categories = {category.text.split("\n")[2].strip(): category.get("href") for category in soup.select(".category-list > li > a")}
categories

{'3C · 家電': '/browse?category=11',
 'DIY 材料': '/browse?category=8',
 '品味美食': '/browse?category=10',
 '媽咪寶寶': '/browse?category=4',
 '寵物用品': '/browse?category=6',
 '居家良品': '/browse?category=5',
 '戶外生活': '/browse?category=9',
 '文具卡片': '/browse?category=3',
 '衣著良品': '/browse?category=0',
 '配件飾品': '/browse?category=2',
 '鞋子包包': '/browse?category=1'}

In [6]:
subcategories_list = soup.select(".subcategory > li > a")
subcategories = {a.text.split('\n')[1].strip(): a.get("href") for a in subcategories_list if re.match("/browse\?category=\d*&subcategory=\d*", a.get("href"))}
subcategories

{'Android 週邊': '/browse?category=11&subcategory=1102',
 'DIY 材料包': '/browse?category=8&subcategory=803',
 'USB 隨身碟': '/browse?category=11&subcategory=1105',
 'iPhone 週邊': '/browse?category=11&subcategory=1101',
 '中性連帽外套/T 恤': '/browse?category=0&subcategory=15',
 '中性針織衫/毛衣': '/browse?category=0&subcategory=21',
 '串珠材料': '/browse?category=8&subcategory=801',
 '乾貨': '/browse?category=10&subcategory=1014',
 '便條紙/標籤貼': '/browse?category=3&subcategory=321',
 '兒童家具': '/browse?category=4&subcategory=403',
 '內衣褲': '/browse?category=0&subcategory=22',
 '公事包': '/browse?category=1&subcategory=112',
 '其他': '/browse?category=9&subcategory=999',
 '冰淇淋/冰棒': '/browse?category=10&subcategory=1009',
 '刊物/書籍': '/browse?category=3&subcategory=313',
 '包裝材料': '/browse?category=8&subcategory=806',
 '化妝包/雜物包': '/browse?category=1&subcategory=113',
 '卡片/明信片': '/browse?category=3&subcategory=304',
 '印章/印台': '/browse?category=3&subcategory=312',
 '吊帶褲': '/browse?category=0&subcategory=13',
 '吊飾': '/browse?catego

In [84]:
# combine categories and subcategories to dictionary of dictionary

product = {}

for index, value in categories.items():
    number = re.findall("\?category=(\d+)", value)
    temp = {}
    for i, v in subcategories.items():
        if re.findall("\?category=(\d+)", v) == number:
            temp[i] = v
    product[index] = temp
product

{'3C · 家電': {'Android 週邊': '/browse?category=11&subcategory=1102',
  'USB 隨身碟': '/browse?category=11&subcategory=1105',
  'iPhone 週邊': '/browse?category=11&subcategory=1101',
  '平板/電腦保護殼': '/browse?category=11&subcategory=1108',
  '捲線器/電線收納': '/browse?category=11&subcategory=1106',
  '擴音器/喇叭': '/browse?category=11&subcategory=1111',
  '相機': '/browse?category=11&subcategory=1110',
  '耳機': '/browse?category=11&subcategory=1107',
  '行動電源/傳輸線': '/browse?category=11&subcategory=1113',
  '觸控筆': '/browse?category=11&subcategory=1109',
  '電話/話筒': '/browse?category=11&subcategory=1112'},
 'DIY 材料': {'DIY 材料包': '/browse?category=8&subcategory=803',
  '串珠材料': '/browse?category=8&subcategory=801',
  '包裝材料': '/browse?category=8&subcategory=806',
  '紙張材料': '/browse?category=8&subcategory=804',
  '線材/綁材': '/browse?category=8&subcategory=805'},
 '品味美食': {'乾貨': '/browse?category=10&subcategory=1014',
  '冰淇淋/冰棒': '/browse?category=10&subcategory=1009',
  '咖啡': '/browse?category=10&subcategory=1018',
  '

In [ ]:
def save_to_csv(urls, category, subcategory, style):
    jd = json.dumps(urls)
    with open("./pinkoi/category{}_subcategory{}_{}.csv".format(category, subcategory, style), "w") as f:
        f.write(jd)

In [63]:
def get_url(URL):
    category = re.findall("\?category=(\d+)", URL)
    subcategory = re.findall("&subcategory=(\d+)", URL)
    style = re.findall("&style=(\w+)", URL)
    switch = True
    counter = 1
    urls = []
    URL = DOMAIN + URL + "&page={}"
    while switch:
        resp = requests.get(URL.format(counter))
        soup = BeautifulSoup(resp.text, "html5lib")
        urls_temp = [a.get("href").split("?")[0] for a in soup.select(".items.clr > div > a")]
        urls.extend(urls_temp)
        verify = soup.select(".g_pagination > a")[-1].text == "下一頁 ►"
        if not verify:
            switch = False
        counter += 1
    save_to_csv(urls, category, subcategory, style)
        

In [ ]:
def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
        for subtitle in product.values():
            for url in subtitle.values():
                for style in style_dict.values():
                    executor.submit(get_url(url+"&style={}".format(style)))

In [ ]:
if __name__=="__main__":
    main()

/browse?category=2&subcategory=205
/browse?category=2&subcategory=203
/browse?category=2&subcategory=201
/browse?category=2&subcategory=202
/browse?category=2&subcategory=204
/browse?category=2&subcategory=217
/browse?category=2&subcategory=208
/browse?category=2&subcategory=207
/browse?category=2&subcategory=210
/browse?category=2&subcategory=215
/browse?category=2&subcategory=220
/browse?category=2&subcategory=218
/browse?category=2&subcategory=219
/browse?category=2&subcategory=224
/browse?category=2&subcategory=216
/browse?category=2&subcategory=222
/browse?category=2&subcategory=226
/browse?category=2&subcategory=227
/browse?category=2&subcategory=209
/browse?category=2&subcategory=225
/browse?category=2&subcategory=213
/browse?category=5&subcategory=508
/browse?category=5&subcategory=506
/browse?category=5&subcategory=505
/browse?category=5&subcategory=511
/browse?category=5&subcategory=528
/browse?category=5&subcategory=520
/browse?category=5&subcategory=539
/browse?category=5&s

In [64]:
#test get_url function using https://www.pinkoi.com/browse/?_currency=TWD&category=2&price=5000&subcategory=205&page={}

x = get_url()
x

['/product/FexVsD5R',
 '/product/MjPQMUqW',
 '/product/EmTG7m8W',
 '/product/YA54cCwq',
 '/product/4Qufk5h3',
 '/product/cDiNXqJ9',
 '/product/4BPRZEtY',
 '/product/2KuhjXVq',
 '/product/7Ae5v3Ws',
 '/product/1J5bl08T',
 '/product/n4Yr58rC',
 '/product/JPAPEZaf',
 '/product/6uRirWKt',
 '/product/sYczCvwH',
 '/product/CNjW9DZE',
 '/product/JP4Rc2fj',
 '/product/1ncgATbK',
 '/product/1a3Z0wR1',
 '/product/mXhLgkkt',
 '/product/eRZmqRFp',
 '/product/Yt8veLeq',
 '/product/NYN7FMVx',
 '/product/1VpBFAvu',
 '/product/1XZgXxKO',
 '/product/i9nECvET',
 '/product/1eAwrHJh',
 '/product/buy72Qrn',
 '/product/1qBQ8Ksx',
 '/product/1SJnVQUY',
 '/product/1kYThNz9',
 '/product/A5GpEdUP',
 '/product/ugeLcSEW',
 '/product/2Bjf6rtX',
 '/product/1XBoneXB',
 '/product/Zgn8Lb4V',
 '/product/xZNf5Tmq',
 '/product/1tghsdSj',
 '/product/RgSyzCjk',
 '/product/xQupV6Ny',
 '/product/W7bga4i6',
 '/product/1_xKfy7R',
 '/product/gPAtBYKc',
 '/product/bxy2q6Th',
 '/product/FWbNG5A7',
 '/product/1befBioW',
 '/product